# 读取数据

## 提取基差数据并进行差分

In [3]:
import new_factor as nf
import pandas as pd

myfactor = nf.NewFactor()
df_basis = myfactor.get_data('basis')
df_basis['T_diff'] = df_basis['T'].diff()
df_basis['TF_diff'] = df_basis['TF'].diff()
df_basis['TS_diff'] = df_basis['TS'].diff() 
df_basis['TL_diff'] = df_basis['TL'].diff()

df_basis.to_excel(f"factor/basis.xlsx")

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


## 读取因子数据

In [3]:
import data
data.run_data()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.
当前工作路径: d:\mycodelife\workshop\yinhe\daily_factor
类别: 利率利差
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/利率利差.xlsx

类别: 国债期货技术指标
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/国债期货技术指标.xlsx

类别: 资金面
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/资金面.xlsx

类别: 国债期货价差
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/国债期货价差.xlsx

类别: 股市
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: ./data/股市.xlsx

            CLOSE
2013-03-31    7.9
2013-06-30    7.6
2013-09-30    7.9
2013-12-31    7.7
2014-03-31    7.6
2014-06-30    7.7
2014-09-30    7.3
2014-12-31    7.4
2015-03-31    7.1
2015-06-30    7.1
2015-09-30    7.0


In [1]:
import factor 
factor.run_factor()

类别: 利率利差
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/利率利差_20250924.xlsx

类别: 国债技术指标
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/国债技术指标_20250924.xlsx

类别: 国债期货技术指标
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/国债期货技术指标_20250924.xlsx

类别: 资金面
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/资金面_20250924.xlsx

类别: 国债期货价差
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/国债期货价差_20250924.xlsx

类别: 股市
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/股市_20250924.xlsx

类别: 宏观
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/宏观_20250924.xlsx

类别: 交易日期
保存并更新数据成功
时间: 20170101 - 20250924
当前存在数据时间范围: 20170101 - 20250924
保存路径: factor/交易日期_20250924.xlsx



In [4]:
import os
import re
import pandas as pd

def merge_excels_by_first_col(folder_path, output_file="merged.xlsx"):
    # 排除：以8位日期开头、紧跟“-因子清单”的文件（不含扩展名匹配）
    pattern = re.compile(r"^\d{8}-因子清单")

    files = [
        f for f in os.listdir(folder_path)
        if f.endswith(".xlsx")
        and not f.startswith("~")
        and not pattern.match(os.path.splitext(f)[0])
    ]
    if not files:
        print("未找到任何可用的 .xlsx 文件")
        return

    dfs = []
    for f in sorted(files):
        path = os.path.join(folder_path, f)
        try:
            # 第一行是列名，第一列作为索引（合并键）
            df = pd.read_excel(path, header=0, index_col=0)
        except Exception as e:
            print(f"读取失败：{path}，原因：{e}")
            continue

        # 规范化索引（第一列），避免空白/类型不一致
        df.index = df.index.astype(str).str.strip()

        # 列名加前缀（保留原始列名，避免冲突）
        stem = os.path.splitext(f)[0]
        df.columns = [f"{stem}__{col}" for col in df.columns]

        dfs.append(df)

    if not dfs:
        print("没有成功读取的表格")
        return

    # 以索引（第一列）按“交集”对齐拼接
    merged = pd.concat(dfs, axis=1, join="inner")

    # 索引转为普通列，并命名为 "date"
    merged = merged.reset_index().rename(columns={"index": "date"})

    # 导出
    merged.to_excel(output_file, index=False, header=True)
    print(f"合并完成，结果已保存到 {output_file}")
    print(f"合并后形状：{merged.shape}（行=共有键数量，列=1+各文件数据列总数）")

# 用法
merge_excels_by_first_col(r"factor", output_file="merged.xlsx")


合并完成，结果已保存到 merged.xlsx
合并后形状：(2122, 771)（行=共有键数量，列=1+各文件数据列总数）


In [9]:
import numpy as np
import pandas as pd

df_raw = pd.read_excel('merged.xlsx')
df_raw.set_index('date', inplace=True)

# 涨跌标签：下一期比当前期涨为 1，跌为 -1，不变为 0
df_raw['label'] = (df_raw['basis__TS'].shift(-1) - df_raw['basis__TS']).apply(
    lambda x: 1 if x > 0 else -1 if x < 0 else 0
)
# 删除最后一行（因为没有未来数据做标签）
df_clean = df_raw.iloc[:-1]

In [10]:
# 假设 df_clean 已经准备好，索引为时间或顺序

label_col = "label"
split_point = int(len(df_clean) * 0.8)

# 按顺序切分
df_train = df_clean.iloc[:split_point]
df_test  = df_clean.iloc[split_point:]

# 拆分特征和标签
X_train, y_train = df_train.drop(columns=[label_col]), df_train[label_col]
X_test,  y_test  = df_test.drop(columns=[label_col]),  df_test[label_col]

train_data = pd.concat([X_train,y_train],axis=1)
test_data = pd.concat([X_test, y_test],axis = 1)

In [14]:
from autogluon.tabular import TabularDataset, TabularPredictor
predictor = TabularPredictor(label=label_col).fit(train_data)

No path specified. Models will be saved in: "AutogluonModels\ag-20250926_041940"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.11.13
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.26100
CPU Count:          16
Memory Avail:       6.51 GB / 15.73 GB (41.4%)
Disk Space Avail:   79.16 GB / 274.71 GB (28.8%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets. Defaulting to `'medium'`...
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='extreme' : New in v1.4: Massively better than 'best' on datasets <30000 samples by using new models meta-learned on https://tabarena.ai: TabPFNv2, TabICL, Mitra, and TabM. Absolute best accuracy. Requires a GPU. Recommended 64 GB CPU memory and 32+ GB GPU memory.
	presets='best'    : Maximize accu

In [15]:
predictor.evaluate(test_data)

{'accuracy': 0.5247058823529411,
 'balanced_accuracy': np.float64(0.5338725424932321),
 'mcc': 0.07409372538778082}

In [13]:
predictor.path

'd:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\ag-20250926_024054'

In [12]:
predictor = TabularPredictor.load("d:\\mycodelife\\workshop\\yinhe\\daily_factor\\AutogluonModels\\best_T")